In [ ]:
from manim import *

In [ ]:
class Current(VGroup):
    def __init__(
        self, point: Sequence[float] = ORIGIN, magnitude=1, direction=OUT, **kwargs
    ):
        if np.all(direction == OUT) or np.all(direction == IN):
            self.direction = direction
        else:
            raise ValueError("only IN and OUT are supported.")
        self.magnitude = magnitude
        if np.all(direction == IN):
            label = VGroup(
                Line(ORIGIN, UR).move_to(ORIGIN),
                Line(ORIGIN, UL).move_to(ORIGIN),
            )
            self.magnitude *= -1
        else:
            label = Dot(radius=0.2)
        super().__init__(**kwargs)
        self.add(Circle(color=WHITE), label).scale(0.2).shift(point)


class MyField(ArrowVectorField):
    def __init__(self, *currents: Current, **kwargs):
        super().__init__(lambda p: self.field_func(p, *currents), **kwargs)

    def field_func(self, p, *currents):
        direction = np.zeros(3)
        pos = []
        for current in currents:
            p0 = current.get_center()
            pos.append(p0)
            mag = current.magnitude
            x, y, z = p - p0
            
            dist = (x ** 2 + y ** 2) ** 1.5
            if any((p - p0) ** 2 > 0.05):
                direction += mag * np.array([- y / dist, x / dist, 0])

        return direction

In [ ]:
%%manim -qh -v WARNING MyExampleScene
from manimlib import *
from manim import *

class MyExampleScene(Scene):         
    def construct(self):
        current1 = Current(LEFT * 2.5)
        current2 = Current(RIGHT * 2.5, direction=IN)
        field = MyField(current1, current2)
        self.add(field, current1, current2)
        
        func = lambda p: field.field_func(p)
        
#         stream_lines = StreamLines(func, stroke_width=3, max_anchors_per_line=30)
#         self.add(stream_lines)
#         stream_lines.start_animation(warm_up=False, flow_speed=1.5)
#         self.wait(stream_lines.virtual_time / stream_lines.flow_speed)

        stream_lines = StreamLines(
            func,         
            dt=0.2,
            virtual_time=10,
            stroke_width=3,
            stroke_color=GREY_A,
            )
        animated_lines = AnimatedStreamLines(stream_lines)
            
        self.add(animated_lines)
        self.wait(5)